In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [3]:
### Load the ANN trained model, scalar pickle, onehotencoder
model=load_model("model.h5")

## load the encoder and scalar
with open("one_hot_encoder_geography.pkl", "rb") as file:
    label_encoder_geo=pickle.load(file)

with open("label_encoder_gender.pkl", "rb") as file:
    label_encoder_gender=pickle.load(file)

with open("scalar.pkl", "rb") as file:
    scaler=pickle.load(file)


In [55]:
# Example input data
input_data={
    'CreditScore': 600,
    'Geography': "France",
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [56]:
# One-hot encode 'Geography'
geo_encoded=label_encoder_geo.transform([[input_data["Geography"]]])
geo_encoded_df=pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(["Geography"]))

/Users/chandanjha/Documents/ann_classification/cenv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [57]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [58]:
## convert input data in the form od dataframe
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [59]:
input_df = pd.DataFrame([input_data])


In [60]:
# Combine one-hot encoded  columns with imput data
input_df = pd.concat([input_df.reset_index(drop=True), geo_encoded_df], axis=1)

In [61]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [62]:
input_df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [63]:
## Encode categorical input variable
input_df["Gender"] = label_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [64]:
## concatenation with onehotencoding data
input_df=input_df.drop("Geography", axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [65]:
## Scaling the input data
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [66]:
## scaled the data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [67]:
## Predict the churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


array([[0.03765783]], dtype=float32)

In [68]:
prediction_probability=prediction[0][0]
prediction_probability

np.float32(0.03765783)

In [69]:
if prediction_probability >  0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
